# Scaling Analysis Examples TTbar70

Authors: Brain Gravelle and Boyana Norris


All this is using the taucmdr python libraries from paratools
http://taucommander.paratools.com/

<a href='#top'>top</a><br>
<a href='#data'>Loading Data</a><br>
<a href='#metrics'>Metrics</a><br>
<a href='#corr'>Correlations</a><br>
<a href='#stl_sb'>Store Buffer Stalls</a><br>
<a href='#fetch_stalls'>Fetch Stalls</a><br>
<a href='#l3'>l3 misses</a><br>

## Imports
This section imports necessary libraies, the metrics.py and utilities.py files and sets up the window.


<a id='top'></a>

In [29]:
# A couple of scripts to set the environent and import data from a .tau set of results
from utilities import *
from metrics import *
# Plotting, notebook settings:
%matplotlib inline  
#plt.rcParams.update({'font.size': 16})
import numbers
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.float_format', lambda x: '%.2e' % x)
pd.set_option('display.max_columns',100)
pd.set_option('max_colwidth', 70)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import copy, shelve

In [30]:
application = "tau_mictest"
exp1 = "TAU__SKL-SP_CMSSW_TTbar_PU70_NVU16_NTH1_HS100"
exp2 = "TAU__SKL-SP_CMSSW_TTbar_PU70_NVU16_NTH1_HS9"

HS100 = load_perf_data(application,exp1,nolibs=False,scaling=False,callpaths=True)
#original_metric_data = remove_erroneous_threads(original_metric_data,  [4, 8, 16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192, 208, 224, 240, 256])

HS9   = load_perf_data(application,exp2,nolibs=False,scaling=False,callpaths=True)

print HS100.keys()
print HS9.keys()


['PAPI_SP_OPS', 'METADATA']
['PAPI_SP_OPS', 'METADATA']


In [31]:
hs100_sorted = HS100['PAPI_SP_OPS'][['Exclusive']].sort_values(by='Exclusive',ascending=False)
hs100_sorted.head(5)

Exclusive
rank context thread region                                                                             
0    0       0      [SUMMARY] .TAU application  => [CONTEXT] .TAU application  => [SAMP...  78932581476
                    [SUMMARY] __svml_sincosf16_z0                                           78932581476
                    [SUMMARY] .TAU application  => [CONTEXT] .TAU application  => [SAMP...  78695262863
                    [SUMMARY] void _INTERNAL2f0d72f6::helixAtRFromIterativeCCS_impl<Mat...  78695262863
                    [SUMMARY] .TAU application  => [CONTEXT] .TAU application  => [SAMP...  73395510875

In [32]:
hs9_sorted = HS9['PAPI_SP_OPS'][['Exclusive']].sort_values(by='Exclusive',ascending=False)
hs9_sorted.head(5)

Exclusive
rank context thread region                                                                             
0    0       0      [SUMMARY] void _INTERNAL2f0d72f6::helixAtRFromIterativeCCS_impl<Mat...  83048373841
                    [SUMMARY] .TAU application  => [CONTEXT] .TAU application  => [SAMP...  83048373841
                    [SUMMARY] .TAU application  => [CONTEXT] .TAU application  => [SAMP...  55469670444
                    [SUMMARY] __svml_sincosf16_z0                                           55469670444
                    [SUMMARY] .TAU application  => [CONTEXT] .TAU application  => [SAMP...  43355015392

In [36]:
METRIC='PAPI_SP_OPS'
func = 'KHC'

diff = hs100_sorted - hs9_sorted
diff_sorted = diff.sort_values(by='Exclusive',ascending=False)
diff_sorted.head(20)

Exclusive
rank context thread region                                                                           
0    0       0      [SUMMARY] __svml_ssincos_cout_rare_internal                              3.00e+10
                    [SUMMARY] .TAU application  => [CONTEXT] .TAU application  => [SAMP...   3.00e+10
                    [SUMMARY] __svml_sincosf16_z0                                            2.35e+10
                    [SUMMARY] .TAU application  => [CONTEXT] .TAU application  => [SAMP...   2.35e+10
                    [SUMMARY] .TAU application  => [CONTEXT] .TAU application  => [SAMP...   1.42e+10
                    [SUMMARY] (anonymous namespace)::MultHelixProp(Matriplex::Matriplex...   1.42e+10
                    [SUMMARY] .TAU application  => [CONTEXT] .TAU application  => [SAMP...   1.23e+10
                    [SUMMARY] (anonymous namespace)::MultHelixPropTransp(Matriplex::Mat...   1.23e+10
                    [SUMMARY] Matriplex::MatriplexSym<float, 6, 16>::SlurpIn(char const...   1.22e+10
                    [SUMMARY] .TAU application  => [CONTEXT] .TAU application  => [SAMP...   1.22e+10
                    [SUMMARY] std::sqrt(float)                                               9.98e+09
                    [SUMMARY] .TAU application  => [CONTEXT] .TAU application  => [SAMP...   9.98e+09
                    [SUMMARY] .TAU application  => [CONTEXT] .TAU application  => [SAMP...   9.15e+09
                    [SUMMARY] Matriplex::MatriplexSym<float, 6, 16>::CopyOut(int, float...   9.15e+09
                    [SUMMARY] Matriplex::MatriplexSym<float, 6, 16>::Subtract(Matriplex...   6.68e+09
                    [SUMMARY] .TAU application  => [CONTEXT] .TAU application  => [SAMP...   6.68e+09
                    [SUMMARY] .TAU application  => [CONTEXT] .TAU application  => [SAMP...   6.17e+09
                    [SUMMARY] mkfit::Track::Track(mkfit::Track const&)                       6.17e+09
                    [SUMMARY] .TAU application  => [CONTEXT] .TAU application  => [SAMP...   4.67e+09
                    [SUMMARY] Matriplex::Matriplex<float, 6, 6, 16>::operator()(int, in...   4.67e+09

In [37]:
diff_searched = get_func_level_metric(diff, func=func, avg=True)
diff_searched.head(20)

,Exclusive
region,
"[SUMMARY] .TAU application => [CONTEXT] .TAU application => [SAMPLE] (anonymous namespace)::KHC(Matriplex::Matriplex<float, 6, 6, 16> const&, Matriplex::MatriplexSym<float, 6, 16> const&, Matriplex::MatriplexSym<float, 6, 16>&)",1.40e+09
"[SUMMARY] (anonymous namespace)::KHC(Matriplex::Matriplex<float, 6, 6, 16> const&, Matriplex::MatriplexSym<float, 6, 16> const&, Matriplex::MatriplexSym<float, 6, 16>&)",1.40e+09


In [38]:
hs9_searched = get_func_level_metric(hs9_sorted, func=func, avg=True)
hs9_searched.head(20)

,Exclusive
region,
"[SUMMARY] (anonymous namespace)::KHC(Matriplex::Matriplex<float, 6, 6, 16> const&, Matriplex::MatriplexSym<float, 6, 16> const&, Matriplex::MatriplexSym<float, 6, 16>&)",14551325140
"[SUMMARY] .TAU application => [CONTEXT] .TAU application => [SAMPLE] (anonymous namespace)::KHC(Matriplex::Matriplex<float, 6, 6, 16> const&, Matriplex::MatriplexSym<float, 6, 16> const&, Matriplex::MatriplexSym<float, 6, 16>&)",14551325140


In [39]:
hs100_searched = get_func_level_metric(hs100_sorted, func=func, avg=True)
hs100_searched.head(20)

,Exclusive
region,
"[SUMMARY] .TAU application => [CONTEXT] .TAU application => [SAMPLE] (anonymous namespace)::KHC(Matriplex::Matriplex<float, 6, 6, 16> const&, Matriplex::MatriplexSym<float, 6, 16> const&, Matriplex::MatriplexSym<float, 6, 16>&)",15946816868
"[SUMMARY] (anonymous namespace)::KHC(Matriplex::Matriplex<float, 6, 6, 16> const&, Matriplex::MatriplexSym<float, 6, 16> const&, Matriplex::MatriplexSym<float, 6, 16>&)",15946816868
